In [2]:
import requests
import pandas as pd

DEFILLAMA_YIELDS_URL = "https://yields.llama.fi/pools"


def fetch_all_defillama_yields() -> pd.DataFrame:
    resp = requests.get(DEFILLAMA_YIELDS_URL, timeout=40, headers={"User-Agent": "defillama-yields-script"})
    resp.raise_for_status()
    payload = resp.json()
    df = pd.DataFrame.from_records(payload["data"])
    return df


# it appears I can just get all ofthem
df = fetch_all_defillama_yields()
df

,chain,project,symbol,tvlUsd,apyBase,apyReward,apy,rewardTokens,pool,apyPct1D,...,sigma,count,outlier,underlyingTokens,il7d,apyBase7d,apyMean30d,volumeUsd1d,volumeUsd7d,apyBaseInception
0,Ethereum,lido,STETH,29685977019,2.62400,NaN,2.62400,None,747c1d2a-c668-4682-b9f9-296708a3dd90,0.04400,...,0.05275,1257,False,[0x0000000000000000000000000000000000000000],NaN,NaN,2.60433,NaN,NaN,NaN
1,Ethereum,binance-staked-eth,WBETH,11445843849,2.69516,NaN,2.69516,None,80b8bf92-b953-4c20-98ea-c9653ef2bb98,0.05256,...,0.03140,878,False,[0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2],NaN,NaN,2.72577,NaN,NaN,NaN
2,Ethereum,ether.fi-stake,WEETH,8983924480,2.12388,0.25422,2.37809,[0x8F08B70456eb22f6109F57b8fafE862ED28E6040],46bd2bdf-6d92-4066-b482-e885ee172264,-0.72726,...,0.10444,525,False,[0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2],NaN,2.71255,3.04428,NaN,NaN,NaN
3,Ethereum,aave-v3,WEETH,5938775249,0.00041,NaN,0.00041,None,db678df9-3281-4bc2-a8bb-01160ffd6d48,0.00000,...,0.00737,577,False,[0xCd5fE23C85820F7B72D0926FC9b05b43E359b7ee],NaN,NaN,0.00041,NaN,NaN,NaN
4,Ethereum,aave-v3,WSTETH,4765377266,0.01906,NaN,0.01906,None,e6435aae-cbe9-4d26-ab2c-a4d533db9972,0.00006,...,0.00252,1007,False,[0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0],NaN,NaN,0.02029,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18212,Optimism,beefy,KWENTA-ETH,10000,NaN,NaN,21.80615,None,2ce63a30-031e-43d9-906f-6d7c270bd994,1.07034,...,1.33855,784,True,"[0x920Cf626a271321C151D027030D5d08aF699456b, 0...",NaN,NaN,15.73300,NaN,NaN,NaN
18213,Base,teller,USDC,10000,17.95716,NaN,17.95716,None,92be1b7a-573c-4d8d-9d18-c17a617678f5,NaN,...,0.30116,23,False,"[0x833589fcd6edb6e08f4c7c32d4f71b54bda02913, 0...",NaN,NaN,14.08404,NaN,NaN,NaN
18214,Sonic,equalizer-exchange,WS-TOONA,10000,NaN,47.30225,47.30225,[0xddf26b42c1d903de8962d3f79a74a501420d5f19],c3cf81fc-ca27-421b-999d-199e657b789e,NaN,...,1.11070,249,False,"[0x039e2fB66102314Ce7b64Ce5Ce3E5183bc94aD38, 0...",NaN,NaN,24.05655,NaN,NaN,NaN
18215,Etherlink,superlend,MTBILL,10000,1.36361,0.00000,1.36361,[],60b238dc-1f4c-4ab1-8889-42a17a4c3cf4,NaN,...,0.55740,205,False,[0xDD629E5241CbC5919847783e6C96B2De4754e438],NaN,NaN,1.47043,NaN,NaN,NaN


In [ ]:
df.columns # underlyingTokens 


Index(['chain', 'project', 'symbol', 'tvlUsd', 'apyBase', 'apyReward', 'apy',
       'rewardTokens', 'pool', 'apyPct1D', 'apyPct7D', 'apyPct30D',
       'stablecoin', 'ilRisk', 'exposure', 'predictions', 'poolMeta', 'mu',
       'sigma', 'count', 'outlier', 'underlyingTokens', 'il7d', 'apyBase7d',
       'apyMean30d', 'volumeUsd1d', 'volumeUsd7d', 'apyBaseInception'],
      dtype='object')

In [31]:
df["TVL"] = df["tvlUsd"].map(lambda x: f"${x/1_000_000:,.2f}M")

interesting_cols = [
    "project",
    "symbol",
    'TVL',
    "apy",
    "pool",
    "apyPct30D",
    "apyMean30d",
    'underlyingTokens'
]

sub_df = df[
    (df["chain"] == "Ethereum")
    & (df["tvlUsd"] >= 1_000_000)
    # & (df["exposure"] == "single")
    & (df["stablecoin"] == False)
    & (df["apyPct30D"] > 10)
]
sub_df[interesting_cols]

,project,symbol,TVL,apy,pool,apyPct30D,apyMean30d,underlyingTokens
416,uniswap-v3,LINK-WETH,$35.98M,38.10004,3025b6b3-e95f-490e-ba10-540c9b1b08f9,12.21913,26.75323,"[0x514910771af9ca656af840dff83e8264ecf986ca, 0..."
502,uniswap-v3,UNI-WETH,$27.64M,192.51688,a9ee1b5f-558b-4e1d-a467-16345e64bc25,186.97604,8.39775,"[0x1f9840a85d5af5bf1d1762f925bdaddc4201f984, 0..."
715,fraxlend,SFRXETH,$16.49M,16.61180,d5a5e303-325e-4b01-a2dc-96494eef2f8b,10.91574,17.02010,[0xac3E018457B222d93114458476f3E3416Abbe38F]
722,uniswap-v3,AAVE-WETH,$16.17M,77.65596,472f422f-f809-4764-8169-b3e40e532bb3,35.49103,43.58204,"[0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9, 0..."
1136,uniswap-v3,INST-WETH,$8.17M,55.50018,ab8c01f0-0586-4077-bae8-86b3e05704c3,50.88595,15.42625,"[0x6f40d4a6237c257fff2db00fa0510deeecd303eb, 0..."
1207,uniswap-v3,WBTC-BADGER,$7.40M,13.82143,996c6ebd-5814-4d7f-970e-b8de7860c4ca,10.66063,8.02134,"[0x2260fac5e5542a773aa44fbcfedf7c193bc2c599, 0..."
1363,uniswap-v3,USDC-SEI,$5.86M,106.48165,5e9db4c8-e4c7-4e0a-8fcb-787e02e8c316,105.36313,59.22652,"[0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48, 0..."
1412,uniswap-v3,LDO-WETH,$5.56M,183.02007,2fc9c593-f66b-4d05-b474-87ebc13ce378,160.84545,34.65772,"[0x5a98fcbea516cf06857215779fd812ca3bef1b32, 0..."
1434,uniswap-v3,WETH-ONDO,$5.47M,62.96254,67958842-74c0-4d3e-b113-5e6cf83893b6,11.77500,19.00896,"[0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2, 0..."
1440,uniswap-v3,UNI-WETH,$5.43M,17.42760,e6922191-75b3-4b30-8258-1a8e09bbe739,17.42591,0.24726,"[0x1f9840a85d5af5bf1d1762f925bdaddc4201f984, 0..."


In [24]:
get_pool = lambda pid: requests.get(f"https://yields.llama.fi/pool/{pid}").json()
sub_df  .iloc[0]["pool"]
# pool_details = get_pool()

'32d65ccd-5c07-4b45-8441-98bc86c0720a'

In [29]:
import requests; data = requests.get("https://yields.llama.fi/chart/32d65ccd-5c07-4b45-8441-98bc86c0720a", timeout=20).json()
data

{'status': 'success',
 'data': [{'timestamp': '2024-07-25T22:03:08.522Z',
   'tvlUsd': 438704,
   'apy': 0,
   'apyBase': 0,
   'apyReward': 0,
   'il7d': None,
   'apyBase7d': None},
  {'timestamp': '2024-07-26T23:01:59.750Z',
   'tvlUsd': 3128950,
   'apy': 12.13151,
   'apyBase': 12.13151,
   'apyReward': 0,
   'il7d': None,
   'apyBase7d': None},
  {'timestamp': '2024-07-27T23:02:10.314Z',
   'tvlUsd': 6028716,
   'apy': 20.38838,
   'apyBase': 20.38838,
   'apyReward': 0,
   'il7d': None,
   'apyBase7d': None},
  {'timestamp': '2024-07-28T23:02:38.519Z',
   'tvlUsd': 8471317,
   'apy': 13.3593,
   'apyBase': 13.3593,
   'apyReward': 0,
   'il7d': None,
   'apyBase7d': None},
  {'timestamp': '2024-07-29T23:01:36.857Z',
   'tvlUsd': 11842309,
   'apy': 26.63577,
   'apyBase': 26.63577,
   'apyReward': 0,
   'il7d': None,
   'apyBase7d': None},
  {'timestamp': '2024-07-30T23:02:46.728Z',
   'tvlUsd': 17125169,
   'apy': 13.1467,
   'apyBase': 13.1467,
   'apyReward': 0,
   'il7d': No

In [30]:
import requests, pandas as pd
pool_id = "32d65ccd-5c07-4b45-8441-98bc86c0720a"
next(p for p in requests.get("https://yields.llama.fi/pools").json()["data"] if p["pool"] == pool_id)


{'chain': 'Ethereum',
 'project': 'morpho-v1',
 'symbol': 'USUALUSDC+',
 'tvlUsd': 62973993,
 'apyBase': 23.33945,
 'apyReward': 0,
 'apy': 23.33945,
 'rewardTokens': ['0x58d97b57bb95320f9a05dc918aef65434969c2b2'],
 'pool': '32d65ccd-5c07-4b45-8441-98bc86c0720a',
 'apyPct1D': 12.02267,
 'apyPct7D': -9.6649,
 'apyPct30D': 15.11189,
 'stablecoin': True,
 'ilRisk': 'no',
 'exposure': 'single',
 'predictions': {'predictedClass': 'Down',
  'predictedProbability': 86,
  'binnedConfidence': 3},
 'poolMeta': None,
 'mu': 10.53366,
 'sigma': 0.27591,
 'count': 472,
 'outlier': False,
 'underlyingTokens': ['0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'],
 'il7d': None,
 'apyBase7d': None,
 'apyMean30d': 11.41604,
 'volumeUsd1d': None,
 'volumeUsd7d': None,
 'apyBaseInception': None}